In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import os

# Importing datasets
who1 = pd.read_csv("../data/raw/HFADataset/HFA Data (table) part 1.csv", low_memory=False)
who2 = pd.read_csv("../data/raw/HFADataset/HFA Data (table) part 2.csv", low_memory=False)
who3 = pd.read_csv("../data/raw/HFADataset/HFA Data (table) part 3.csv", low_memory=False)
who = pd.concat([who1, who2, who3], axis=0)

label = pd.read_excel("../data/raw/HFADataset/HFA Metadata.xlsx", sheet_name="Labels", header=1)

eu_member = pd.read_excel("../data/raw/HFADataset/HFA Metadata.xlsx", sheet_name="Country groups mapping", header=1)
eu_member = eu_member[['Country short name','Unnamed: 1','EU_MEMBERS']]
eu_member.columns = ['Country short name','COUNTRY_REGION','EU_MEMBERS']
eu_countries = eu_member.COUNTRY_REGION[eu_member['EU_MEMBERS']=='yes']

pop = pd.read_csv("../data/raw/WB/API_SP.POP.TOTL_DS2_en_csv_v2_3401680.csv", low_memory=False, header=2)
pop = pop[['Country Name', 'Country Code','2018']]
pop.columns = ['Country Name', 'COUNTRY_REGION', 'Population']

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [2]:
# Collecting relevant variables

# Mental disorders
#HFA_387	Incidence of mental disorders per 100 000
#HFA_388	Number of new cases of mental disorders

# GDP
# HFA_32	Gross national income, US$ per capita
# HFA_33	Gross domestic product (GDP), US$ per capita

variables = ["HFA_387", "HFA_388", "HFA_32", "HFA_33"]
data = who[(who['Measure code'].isin(variables)) & (who['COUNTRY_REGION'].isin(eu_countries))]

In [37]:
# Data Visualization

# Creating a line plot with multiple lines (EU Countries)

#HFA_387	Incidence of mental disorders per 100 000 by EU Countries
HFA_387 = data[(data['Measure code']=="HFA_387")]

fig = px.line(HFA_387, x='YEAR', y='VALUE', color='COUNTRY_REGION', title='Incidence of mental disorders per 100 000')

# Show the figure
fig.write_html('first_figure.html', auto_open=True)
fig.write_image("../figures/mental_eu.png",)

In [44]:
print('Incidence of mental disorders per 100 000')
data[["COUNTRY_REGION","VALUE"]][(data["Measure code"]=="HFA_387") & (data["YEAR"]==2020)].sort_values(by="VALUE", ascending=False).head()

Incidence of mental disorders per 100 000


,COUNTRY_REGION,VALUE
320185,LTU,5205.41
319854,EST,1586.03
319778,DEU,1463.54
320392,SVK,1118.97
320320,ROU,1058.57


In [39]:
print('Number of new cases of mental disorders')
data[["COUNTRY_REGION","VALUE"]][(data["Measure code"]=="HFA_388") & (data["YEAR"]==2020)].sort_values(by="VALUE", ascending=False).head().round()

Number of new cases of mental disorders


,COUNTRY_REGION,VALUE
320765,DEU,1177788.0
321247,POL,360402.0
321275,ROU,234521.0
321010,ITA,232376.0
321135,LTU,145472.0


In [46]:
# Create a bubble chart (GDP & Mental disorder & Population)

HFA_33 = data[(data['Measure code']=="HFA_33")]
HFA_33 = HFA_33[['COUNTRY_REGION','YEAR','VALUE']]
HFA_33.columns = ['COUNTRY_REGION','YEAR','GDP_per_Capita']
HFA_33['ln_GDP_per_Capita'] = HFA_33['GDP_per_Capita'].apply(lambda x: np.log(x))
GDP_mental=pd.merge(HFA_33, HFA_387, on=["COUNTRY_REGION","YEAR"])
GDP_mental=GDP_mental.drop(["Measure code","SEX","PLACE_RESIDENCE","YES_NO"], axis=1)
GDP_mental.columns = ["COUNTRY_REGION", "YEAR", "GDP_per_Capita","ln_GDP_per_Capita","Mental_disorder"]
GDP_mental = GDP_mental[GDP_mental["YEAR"]==2018]
GDP_mental_pop=pd.merge(GDP_mental, pop, on=["COUNTRY_REGION"])

fig = px.scatter(GDP_mental_pop,x="GDP_per_Capita", y="Mental_disorder", color="Country Name",title='GDP and Mental disorders',
                 size="Population", hover_name="Country Name", log_x=True,size_max=50)

#fig.update_layout(font=dict(size=26))
fig.write_html('first_figure.html', auto_open=True)
#fig.show()
fig.write_image("../figures/mental_GDP_pop.png")

In [121]:
# Create a scatter plot (GNI & Mental disorder)

HFA_32 = data[(data['Measure code']=="HFA_32")]
HFA_32 = HFA_32[['COUNTRY_REGION','YEAR','VALUE']]
HFA_32.columns = ['COUNTRY_REGION','YEAR','GNI_per_Capita']
HFA_32['ln_GNI_per_Capita'] = HFA_32['GNI_per_Capita'].apply(lambda x: np.log(x))
GNI_mental=pd.merge(HFA_32, HFA_387, on=["COUNTRY_REGION","YEAR"])
GNI_mental=GNI_mental.drop(["Measure code","SEX","PLACE_RESIDENCE","YES_NO"], axis=1)
GNI_mental.columns = ["COUNTRY_REGION", "YEAR", "GNI_per_Capita","ln_GNI_per_Capita","Mental_disorder"]

fig = px.scatter(GNI_mental.query("YEAR==2020"),x="GNI_per_Capita", y="Mental_disorder", color='COUNTRY_REGION', log_x=True)
fig.write_html('first_figure.html', auto_open=True)
#fig.show()